# HyperLogLog

`HyperLogLog` -  вероятностный алгоритм для оценки уникальных элементов в больших наборах данных. Он обеспечивает более эффективное использование памяти по сравнению с точными методами, такими как подсчет уникальных элементов. Основная идея заключается в использовании хэш-функций для преобразования элементов в битовые строки и разделении их на группы по самым длинным последовательностям нулей. Затем используется оценочная функция для вычисления приблизительного количества уникальных элементов на основе длин этих последовательностей. Алгоритм `HyperLogLog` обеспечивает точность оценки с уровнем погрешности, который можно контролировать с помощью параметра b, который определяет количество бит для регистров.

In [9]:
import numpy as np

from typing import Any


class HyperLogLog:
    def __init__(self, b: int):
        self.m = b ** 2
        self.b = b
        self.registers = np.zeros(self.m) 
        self.alpha = self.calculate_alpha(self.m)

    @staticmethod
    def calculate_alpha(m: int) -> float:
        if m == 16:
            return 0.673
        elif m == 32:
            return 0.697
        elif m == 64:
            return 0.709
        else:
            return 0.7213 / (1 + 1.079 / m)

    def add(self, element: Any):
        hash_value = hash(element)
        ## очень и очень неэффективно
        hash_bin = format(hash_value, "064b")

        reg_index = int(hash_bin[:self.b], 2)

        leading_zeros = hash_bin[self.b:].index('1') + 1
        self.registers[reg_index] = max(self.registers[reg_index], leading_zeros)

    def count(self) -> int:
        total = (2 ** (-self.registers)).sum()
        estimate = self.alpha * (self.m ** 2) / total

        # Bias correction
        if estimate <= 5/2 * self.m:
            zeros_count = (self.registers == 0).sum()
            if zeros_count != 0:
                corrected_estimate = self.m * np.log(self.m / zeros_count)
                return int(corrected_estimate)
        
        return int(estimate)

In [13]:
import random

hll = HyperLogLog(5)
elements = random.choices([
    "apple", "orange", "raspberry"], 
    k=10000
)
for x in elements:
    hll.add(x)

hll.count()

3

# BloomFilter

`BloomFilter` - это вероятностная структура данных, используемая для эффективного определения наличия элемента в множестве. Она состоит из битового массива определённого размера и нескольких хэш-функций. При добавлении элемента в фильтр, элемент хэшируется несколькими функциями, и биты в соответствующих позициях массива устанавливаются в 1. При проверке наличия элемента, элемент снова хэшируется этими функциями, и если все соответствующие биты установлены в 1, элемент вероятно присутствует в фильтре. Cуществует вероятность ложноположительных ответов.

In [23]:
import numpy as np
from typing import Callable, List, Any

class BloomFilter:
    def __init__(self, num_buckets: int, hash_functions: List[Callable[[Any], int]]):
        self.num_buckets = num_buckets
        self.hash_functions = hash_functions
        self.bit_array = np.zeros(num_buckets)

    def add(self, item: Any):
        for hf in self.hash_functions:
            index = hf(item) % self.num_buckets
            self.bit_array[index] = 1

    def __contains__(self, item: Any) -> bool:
        for hf in self.hash_functions:
            index = hf(item) % self.num_buckets
            if self.bit_array[index] == 0:
                return False
        return True

hash_functions = [
    # очень плохие функции
    lambda x: hash(x) * 7,
    lambda x: hash(str(x)) * 11
]


bf = BloomFilter(20, hash_functions)

lst = ["apple", "orange", "raspberry"]
for x in lst:
    bf.add(x)

print("test" in bf)
print("orange" in bf)


(False, True)